# Circulations Observed during NDJF 2015-16

The purpose of this notebook is to plot the climatology of rainfall and circulation patterns, and to plot the anomalies that occurred during NDJF 2015-16.

In [ ]:
import numpy as np
import xarray as xr
import calendar
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import paraguayfloodspy.visualize as viz # custom plotting library
from paraguayfloodspy.xrutil import *

Load up some parameters for plotting

In [ ]:
%run ../config/PlotParameters.py
print(extent)
print(cmap)

Define some additional parameters for plotting

In [ ]:
%matplotlib inline
savefigs = True
months_plot = [11, 12, 1, 2]
years_plot = [2015,2015,2016,2016]

Read in the raw data

In [ ]:
psi850 = xr.open_dataset("../data/derived/psi_850.nc")
psi200 = xr.open_dataset("../data/derived/psi_200.nc")
prcp = xr.open_dataset('../data/derived/precip.nc')

Get the X and Y coordinates for plotting

In [ ]:
X, Y = np.meshgrid(psi850.lon, psi850.lat)
Xp, Yp = np.meshgrid(prcp.lon, prcp.lat)
ncols = len(months_plot)

## Plot Anomalies

In [ ]:
fig, axes = viz.SetupAxes(ncol = ncols, nax = 3 * ncols, proj = ccrs.Orthographic(-60, -10), figsize = [17, 9.5])
for i,m in enumerate(months_plot):
    def selector(ds):
        ds = ds.sel(time = slice('2015-11-01', '2016-02-28'))
        ds = ds.sel(time = (ds['time.month'] == m)).mean(dim='time')
        return(ds)
    #
    ax = axes[0, i]
    ax.set_title('{} {}'.format(calendar.month_name[m], years_plot[i]))
    C0 = ax.contourf(X, Y, selector(psi200['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-2.5e7, 2.5e7, 11))
    #
    ax = axes[1, i]
    C1 = ax.contourf(X, Y, selector(psi850['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-6e6, 6e6, 13))
    #
    ax = axes[2, i]
    sub = selector(prcp['anomaly']).values
    sub = np.ma.masked_invalid(sub)
    C2 = ax.contourf(Xp, Yp, sub, transform = ccrs.PlateCarree(), 
                       cmap = cmap['rain_a'], levels = np.linspace(-9, 9, 10),
                    extend = 'both')
plt.tight_layout()
fig.subplots_adjust(right=0.94)
cax0 = fig.add_axes([0.97, 0.71, 0.01, 0.2])
cax1 = fig.add_axes([0.97, 0.39, 0.01, 0.2])
cax2 = fig.add_axes([0.97, 0.05, 0.01, 0.25])
cbar0 = fig.colorbar(C0, cax = cax0)
cbar0.formatter.set_powerlimits((7, 7))
cbar0.update_ticks() 
cbar0.set_label(r'$\psi_{200}$ Anomaly [$m^2$/s]', rotation=270)
cbar0.ax.get_yaxis().labelpad = 20
#
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.formatter.set_powerlimits((6, 6))
cbar1.update_ticks() 
cbar1.set_label(r'$\psi_{850}$ Anomaly [$m^2$/s]', rotation=270)
cbar1.ax.get_yaxis().labelpad = 20
# 
cbar2 = fig.colorbar(C2, cax=cax2)
cbar2.set_label('Precip. Anomaly [mm/d]', rotation=270)
cbar2.ax.get_yaxis().labelpad = 20
# 
viz.FormatAxes(axes[0,:], extent = extent['SH'])
viz.FormatAxes(axes[1,:], extent = extent['SH'])
viz.FormatAxes(axes[2,:], extent = extent['SAm'])
#
if savefigs:
    fig.savefig("../figs/NDJF201516Anomaly.pdf", bbox_inches='tight')